# Table of Contents
 <p><div class="lev1"><a href="#VARIABLE-DESCRIPTIONS:"><span class="toc-item-num">1&nbsp;&nbsp;</span>VARIABLE DESCRIPTIONS:</a></div>

# VARIABLE DESCRIPTIONS:

* survival        Survival            (0 = No; 1 = Yes)
* pclass          Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
* name            Name
* sex             Sex
* age             Age
* sibsp           Number of Siblings/Spouses Aboard
* parch           Number of Parents/Children Aboard
* ticket          Ticket Number
* fare            Passenger Fare
* cabin           Cabin
* embarked        Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)

SPECIAL NOTES:
Pclass is a proxy for socio-economic status (SES)
 1st ~ Upper; 2nd ~ Middle; 3rd ~ Lower

Age is in Years; Fractional if Age less than One (1)
 If the Age is Estimated, it is in the form xx.5

With respect to the family relation variables (i.e. sibsp and parch)
some relations were ignored.  The following are the definitions used
for sibsp and parch.

Sibling:  Brother, Sister, Stepbrother, or Stepsister of Passenger Aboard Titanic
Spouse:   Husband or Wife of Passenger Aboard Titanic (Mistresses and Fiances Ignored)
Parent:   Mother or Father of Passenger Aboard Titanic
Child:    Son, Daughter, Stepson, or Stepdaughter of Passenger Aboard Titanic

Other family relatives excluded from this study include cousins,
nephews/nieces, aunts/uncles, and in-laws.  Some children travelled
only with a nanny, therefore parch=0 for them.  As well, some
travelled with very close friends or neighbors in a village, however,
the definitions do not support such relations.

In [33]:
# This Python 3 environment comes with many helpful analytics libraries 
# installed
# It is defined by the kaggle/python docker image: 
# https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from IPython.display import display
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score, accuracy_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from matplotlib import pyplot as plt

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) 
# will list the files in the input directory

import sh
print(sh.ls('./input'))

genderclassmodel.csv  gendermodel.csv  test.csv  train.csv  train.csv.ori



In [108]:
Y_test =  pd.read_csv('./input/genderclassmodel.csv')
test =  pd.read_csv('./input/test.csv')
train =  pd.read_csv('./input/train.csv')

print('test set has %s rows and %s columns.' % test.shape)
print('test set has %s rows and %s columns.' % Y_test.shape)
print('train set has %s rows and %s columns.' % train.shape)

test set has 418 rows and 11 columns.
test set has 418 rows and 2 columns.
train set has 891 rows and 12 columns.


In [27]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [87]:
def prepare_df(df, p_unique=0.2):
    df = df.copy()
    
    size = len(df)
    cols = list(df.columns)
    
    for c in cols:
        if df[c].dtype == object:
            if df[c].nunique()/size > p_unique:
                df.drop(c, axis=1, inplace=True)
                continue
                
            df = pd.concat([
                df.drop(c, axis=1), 
                pd.get_dummies(df[c], drop_first=True)
            ], axis=1)
            
    return df

In [124]:
train_ = train.copy()

train_.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
train_.dropna(inplace=True)

X_train = prepare_df(train_.drop('Survived', axis=1))
y_train = train_[['Survived']]

In [125]:
test_ = pd.concat([Y_test, test.iloc[:, 1:]], axis=1)

test_.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test_.dropna(inplace=True)

X_test = prepare_df(test_.drop('Survived', axis=1))
y_test = test_[['Survived']]

In [126]:
test_.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

In [128]:
print('X_test set has %s rows and %s columns.' % X_test.shape)
print('y_test set has %s rows and %s columns.' % y_test.shape)
print('X_train set has %s rows and %s columns.' % X_train.shape)
print('Y_train set has %s rows and %s columns.' % y_train.shape)

X_test set has 331 rows and 8 columns.
y_test set has 331 rows and 1 columns.
X_train set has 712 rows and 8 columns.
Y_train set has 712 rows and 1 columns.


In [129]:
pipeline = make_pipeline(
    RandomForestClassifier()
)

In [134]:
'''
<class 'sklearn.ensemble.forest.RandomForestClassifier'>
f1 score
0.768558951965
precision score
1.0
accuracy score
0.873205741627


<class 'sklearn.svm.classes.SVC'>
f1 score
0.873015873016
precision score
0.990990990991
accuracy score
0.923444976077
'''
for model in [RandomForestClassifier(), SVC()]:
    print('\n%s' % model)
    model.fit(X_train, y_train)

    predicted = model.predict(X_test)
    print('f1 score')
    print(f1_score(y_test, predicted))
    print('precision score')
    print(precision_score(y_test, predicted))
    print('accuracy score')
    print(accuracy_score(y_test, predicted))



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
f1 score
0.761904761904762
precision score
0.732824427480916
accuracy score
0.8187311178247734

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
f1 score
0.4621513944223108
precision score
0.4461538461538462
accuracy score
0.5921450151057401


/home/xmn/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/home/xmn/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
predicted.to_csv('output/svc.csv', index=False)